In [1]:
# Instalar DEAP
!pip install deap -q

import random
from deap import base, creator, tools, algorithms

# ==============================
# DATOS DEL PROBLEMA
# ==============================

# 10 materias con distintos tamaños y frecuencias
nombres_materias = [
    "Cálculo I", "Álgebra I", "Física I", "Química General",
    "Programación", "Estadística", "Fisica II", "Algoritmos Evolutivos",
    "Termodinámica", "Laboratorio", "Aprendizaje por Refuerzo"
]

estudiantes_por_materia = [45, 62, 50, 35, 60, 30, 45, 40, 35, 25, 22]
sesiones_por_materia = [2, 2, 2, 1, 2, 2, 1, 2, 1, 2]  # total = 18 sesiones

# Construir mapeo: posición → materia
sesion_a_materia = []
for i, sesiones in enumerate(sesiones_por_materia):
    sesion_a_materia.extend([i] * sesiones)

print(f"Total de sesiones a programar: {len(sesion_a_materia)}")

# Aulas: ahora 6 aulas con capacidades variadas
capacidad_aulas = [30, 40, 50, 60, 70, 50]  # aula 0 a 5
AULAS = len(capacidad_aulas)

# Horario: lunes a viernes, 4 bloques por día (8–18)
DIAS = 5
BLOQUES = 4  # los bloques son los horarios de clase de 8-10, 10-12, 14-16, 16-18

# ==============================
# CONFIG DE DEAP
# ==============================

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# ==============================
# FUNCIÓN Fitness
# ==============================

def evaluar(individuo):
    penalizacion = 0
    uso_total = 0.0
    ocupados = set()

    for i, (aula, dia, bloque) in enumerate(individuo):
        # Validar rangos
        if not (0 <= aula < AULAS and 0 <= dia < DIAS and 0 <= bloque < BLOQUES):
            return (-10000.0,)

        materia = sesion_a_materia[i]
        num_est = estudiantes_por_materia[materia]
        cap = capacidad_aulas[aula]

        # R1: Capacidad
        if cap < num_est:
            penalizacion += 5000  # penalización alta

        # R2: Solapamiento
        clave = (aula, dia, bloque)
        if clave in ocupados:
            penalizacion += 5000
        else:
            ocupados.add(clave)

        # Acumular uso
        if cap >= num_est:
            uso_total += num_est / cap # <-- aqui esta la sumatoria que decia

    # R3 (opcional): incentivar uso de todos los días
    dias_usados = {dia for (_, dia, _) in individuo}
    if len(dias_usados) < DIAS:
        # Penalización suave por no usar todos los días
        penalizacion += (DIAS - len(dias_usados)) * 10

    fitness = uso_total - penalizacion
    return (fitness,)



def crear_individuo():
    individuo = []
    for _ in range(len(sesion_a_materia)):
        aula = random.randint(0, AULAS - 1)
        dia = random.randint(0, DIAS - 1)
        bloque = random.randint(0, BLOQUES - 1)
        individuo.append((aula, dia, bloque))
    return creator.Individual(individuo)

# ==============================
# OPERADORES GENÉTICOS
# ==============================

def mutar_individuo(individuo, indpb=0.15):
    for i in range(len(individuo)):
        if random.random() < indpb:
            aula, dia, bloque = individuo[i]
            if random.random() < 0.4:
                aula = random.randint(0, AULAS - 1)
            if random.random() < 0.4:
                dia = random.randint(0, DIAS - 1)
            if random.random() < 0.4:
                bloque = random.randint(0, BLOQUES - 1)
            individuo[i] = (aula, dia, bloque)
    return individuo,

def crossover_individuos(ind1, ind2):
    tools.cxTwoPoint(ind1, ind2)  # cruce de dos puntos (mejor para problemas grandes)
    return ind1, ind2

# ==============================
# CONFIG TOOLBOX
# ==============================

toolbox = base.Toolbox()
toolbox.register("individual", crear_individuo)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluar)
toolbox.register("mate", crossover_individuos)
toolbox.register("mutate", mutar_individuo, indpb=0.15)
toolbox.register("select", tools.selTournament, tournsize=5)

# ==============================
# EJECUCION del AG
# ==============================

def main():
    random.seed(123)

    pop = toolbox.population(n=200)   # población más grande
    hof = tools.HallOfFame(3)         # guardar top 3 soluciones
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: sum(v[0] for v in x) / len(x))
    stats.register("min", lambda x: min(v[0] for v in x))
    stats.register("max", lambda x: max(v[0] for v in x))

    print("Iniciando algoritmo genético...\n")

    algorithms.eaSimple(
        pop, toolbox,
        cxpb=0.8,      # más cruce
        mutpb=0.2,     # más mutación
        ngen=120,      # más generaciones
        stats=stats,
        halloffame=hof,
        verbose=True
    )

    # Mostrar las mejores soluciones
    dias_semana = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes"]

    for idx, mejor in enumerate(hof):
        print("\n" + "="*60)
        print(f" SOLUCIÓN {idx+1} (Fitness: {mejor.fitness.values[0]:.3f})")
        print("="*60)

        # Agrupar por día para mejor visualización
        horario_por_dia = {d: [] for d in range(DIAS)}
        for i, (aula, dia, bloque) in enumerate(mejor):
            mat_id = sesion_a_materia[i]
            nombre = nombres_materias[mat_id]
            est = estudiantes_por_materia[mat_id]
            cap = capacidad_aulas[aula]
            hora_inicio = 8 + 2 * bloque
            horario_por_dia[dia].append((hora_inicio, nombre, aula, cap, est))

        # Imprimir por día
        for dia in range(DIAS):
            print(f"\n {dias_semana[dia]}:")
            if not horario_por_dia[dia]:
                print("  (Sin clases)")
            else:
                for hora, nombre, aula, cap, est in sorted(horario_por_dia[dia]):
                    print(f"  {hora}:00–{hora+2}:00 | {nombre:<20} | Aula {aula} (cap={cap}, est={est})")

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 2.3 MB/s eta 0:00:00
Total de sesiones a programar: 17
Iniciando algoritmo genético...

gen	nevals	avg     	min   	max     
0  	200   	-31442.4	-54995	-9987.76
1  	161   	-21065.9	-44991.5	-4988.34
2  	160   	-14165  	-34990.8	-4987.5 
3  	178   	-11164.4	-25000.3	-4986.4 
4  	174   	-7539.11	-19990.8	13.8964 
5  	169   	-5862.76	-19989.6	14.7381 
6  	157   	-3612.34	-19988.9	14.9095 
7  	158   	-1936.68	-14996  	15.3095 
8  	155   	-1136.18	-9987.2 	15.5452 
9  	159   	-935.685	-9986.98	15.5452 
10 	164   	-1235.24	-14988.5	15.7452 
11 	150   	-459.842	-9985.53	15.7452 
12 	164   	-534.583	-14986.3	15.8881 
13 	167   	-709.471	-14986.1	15.8881 
14 	171   	-684.406	-14986  	15.8881 
15 	174   	-334.343	-9985.91	15.8881 
16 	165   	-684.333	-14986.3	15.8881 
17 	143   	-684.291	-19986  	15.8881 
18 	166   	-434.2  	-9986.68	15.8881 
19 	164   	-309.201	-9985.29	15.8881 
20 	178   	-234.21 	-9986.83	15.8881 
21 	163   	-484.183	-